In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
data_dir = '/content/drive/MyDrive/seg_pred'  # change to your folder
print(os.path.exists(data_dir))
print(os.listdir(data_dir)[:30])  # list first 30 entries

In [ ]:
import glob, os
img_paths = glob.glob(os.path.join(data_dir, '**', '*.jpg'), recursive=True)  # jpg example
# or include png:
# img_paths = glob.glob(os.path.join(data_dir, '**', '*.[jp][pn]g'), recursive=True)
len(img_paths)

In [ ]:
import tensorflow as tf

ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,            # directory with subfolders per class
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224),  # adjust
    shuffle=True,
)
for images, labels in ds.take(1):
    print(images.shape, labels.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 123

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # reserve 20% for validation
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

print("Defined train_datagen and val_datagen with validation_split=0.2")

In [ ]:
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='training',
    seed=SEED
)

validation_generator = val_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    subset='validation',
    seed=SEED
)

In [ ]:
batch_images, batch_labels = next(train_generator)
print("Batch images shape:", batch_images.shape)
print("Batch labels shape:", batch_labels.shape)

def show_images(images, cols=6):
    rows = int(np.ceil(len(images) / cols))
    plt.figure(figsize=(cols*2, rows*2))
    for i, img in enumerate(images):
        plt.subplot(rows, cols, i+1)
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Display first 12 augmented images from this batch
show_images(batch_images[:12], cols=6)

### Why data augmentation matters

- Prevents overfitting  
  Augmentation makes many slightly different versions of each image. This stops the model from memorizing the exact training images and helps it learn real patterns.

- Helps the model work on new images  
  Seeing many variations during training teaches the model to handle small changes it will meet in the real world.

- Cheap way to get more data  
  You don’t need to label more images,augmentation creates extra useful examples from what you already have.

Augmentation is an easy, low-cost trick that makes a fine-tuned model more reliable on small datasets.

In [ ]:
# Step 2: load ResNet50 base (no top) and freeze it
input_shape = (224, 224, 3)  # standard for ResNet50

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = False  # freeze the whole convolutional base

# Optional: show how many layers are trainable (should be zero)
trainable_count = sum(1 for layer in base_model.layers if layer.trainable)
print("ResNet50 base layers:", len(base_model.layers))
print("Trainable layers in base:", trainable_count)

In [ ]:
# Step 5: compile the model (ready for training the top-only head)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',  # or 'categorical_crossentropy' if using one-hot labels
    metrics=['accuracy']
)
print("Model compiled. Top layers are trainable; base is frozen.")

### Why we freeze early convolutional layers
- Early layers learn basic visual patterns like edges, colors and simple textures.  
- These basic features are useful across many image tasks, so we keep them fixed to avoid destroying that knowledge.  
- Freezing speeds up training and reduces overfitting when we only have a small dataset.  
- We train only the top (new) layers so the model quickly learns task-specific patterns for the 6 classes.

freeze early layers because they already know useful low-level features and freezing them makes fine-tuning faster and more stable on small datasets.

In [ ]:
import tensorflow as tf
import pathlib
import numpy as np
from sklearn.model_selection import train_test_split

DATA_ROOT = "/content/drive/MyDrive/seg_pred"  # change as needed
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
SEED = 123
TEST_SIZE = 0.1    # fraction for test
VAL_SIZE = 0.1     # fraction for validation (from original dataset)

data_root = pathlib.Path(DATA_ROOT)

# If images are organized as class subfolders:
class_dirs = [d for d in data_root.iterdir() if d.is_dir()]
class_dirs = sorted(class_dirs)
class_names = [d.name for d in class_dirs]
print("Detected classes:", class_names)
name_to_idx = {name: idx for idx, name in enumerate(class_names)}

# Gather all file paths and labels
filepaths = []
labels = []
for class_dir in class_dirs:
    for img_path in class_dir.glob('*'):
        if img_path.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp']:
            filepaths.append(str(img_path))
            labels.append(name_to_idx[class_dir.name])

filepaths = np.array(filepaths)
labels = np.array(labels)

# First split off test set
train_paths, test_paths, train_labels, test_labels = train_test_split(
    filepaths, labels, test_size=TEST_SIZE, stratify=labels, random_state=SEED
)

# Then split train into train and val
relative_val_size = VAL_SIZE / (1.0 - TEST_SIZE)  # adjust because test already removed
train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_paths, train_labels, test_size=relative_val_size, stratify=train_labels, random_state=SEED
)

print(f"Counts -> train: {len(train_paths)}, val: {len(val_paths)}, test: {len(test_paths)}")

# Preprocessing function
def preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_image(image, channels=3, expand_animations=False)
    image = tf.image.convert_image_dtype(image, tf.float32)  # scales to [0,1]
    image = tf.image.resize(image, IMG_SIZE)
    return image, label

# Build tf.data datasets
def build_dataset(paths, labels, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(paths), seed=SEED)
    ds = ds.map(lambda p, l: preprocess_image(p, l), num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds

train_ds = build_dataset(train_paths, train_labels, shuffle=True)
val_ds   = build_dataset(val_paths, val_labels, shuffle=False)
test_ds  = build_dataset(test_paths, test_labels, shuffle=False)  # keep order stable for predictions

# Expose class names
print("Class names:", class_names)

In [ ]:
# Step 9: Train only the top head first (assumes `model` built previously and base frozen)
# If you haven't created `model` yet, copy the ResNet50 + head creation code from earlier steps.
import tensorflow as tf

# Confirm which layers are trainable
trainable_layers = sum(1 for layer in model.layers if layer.trainable)
print("Trainable layers before head training:", trainable_layers)

# Compile with a moderate learning rate for the head
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

EPOCHS_HEAD = 2 # change as needed
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
]

history_head = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_HEAD,
    callbacks=callbacks
)

In [ ]:
# Step 10: Unfreeze last N layers of base_model and fine-tune with a low LR
# Identify the base model; if you used variable name base_model earlier it exists.
N = 30  # number of last layers to unfreeze; tune this
# If base_model variable not available, find it inside the model by name or index:
try:
    base_model
except NameError:
    # attempt to find first layer that has 'resnet' in its name (common)
    for layer in model.layers:
        if 'resnet' in layer.name.lower():
            base_model = layer
            break

# More general: if base_model is a Model, set its layers. If base_model is a layer, find model.layers that are in the base.
if hasattr(base_model, 'layers'):
    for layer in base_model.layers[-N:]:
        layer.trainable = True
else:
    # fallback: unfreeze last N layers of entire model (careful)
    for layer in model.layers[-N:]:
        layer.trainable = True

# Recompile with low learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

EPOCHS_FINE = 2  # change as needed
history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FINE,
    callbacks=callbacks
)

In [ ]:
# Step 11: Basic evaluation (loss + acc)
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test loss: {test_loss:.4f}  Test accuracy: {test_acc:.4f}")

In [ ]:
# Step 12: Generate classification report + confusion matrix together
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Get predictions (probabilities) and true labels
y_prob = model.predict(test_ds, verbose=1)
y_pred = np.argmax(y_prob, axis=1)

# Extract true labels from test_ds (shuffle=False ensured earlier)
y_true = np.concatenate([y.numpy() for _, y in test_ds], axis=0)

# If dataset yields labels as int tensors, the above works.
# Print classification report (text)
# Use only the class names that the model was trained on
report = classification_report(y_true, y_pred, target_names=[class_names[i] for i in np.unique(y_true)], digits=4)
print("Classification Report:\n")
print(report)

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[class_names[i] for i in np.unique(y_true)], yticklabels=[class_names[i] for i in np.unique(y_true)])
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()

# Save figure in case you want a separate image
plt.savefig("/content/confusion_matrix.png", dpi=200)
print("Confusion matrix saved to /content/confusion_matrix.png")

In [ ]:
# Cell 1: compute predictions (if needed), confusion matrix, and identify top confused pair
import numpy as np
from sklearn.metrics import confusion_matrix

# Compute predictions if not already present
try:
    y_pred  # noqa: F821
    print("Using existing y_pred")
except NameError:
    # predict on test_ds (this may take time)
    print("Computing predictions on test dataset...")
    y_prob = model.predict(test_ds, verbose=1)
    y_pred = np.argmax(y_prob, axis=1)

# Get true labels aligned with order of test_paths
try:
    # Prefer using the array used to build test_ds if available
    y_true = np.array(test_labels)  # from the earlier train_test_split
    if len(y_true) != len(y_pred):
        raise ValueError("Length mismatch between test_labels and y_pred")
except Exception:
    # Fallback: extract from test_ds (works if shuffle=False)
    y_true = np.concatenate([y.numpy() for _, y in test_ds], axis=0)

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion matrix shape:", cm.shape)

# Find largest off-diagonal element
cm_off = cm.copy().astype(np.int64)
np.fill_diagonal(cm_off, -1)  # exclude diagonal
max_idx = np.unravel_index(np.argmax(cm_off), cm_off.shape)
max_value = cm_off[max_idx]
class_a_idx, class_b_idx = max_idx

print(f"Most confused pair: True='{class_names[class_a_idx]}'  Predicted='{class_names[class_b_idx]}'  (count = {max_value})")
# Also print the reverse confusion count for reference
reverse_count = cm[class_b_idx, class_a_idx]
print(f"Reverse confusion (True='{class_names[class_b_idx]}' Pred='{class_names[class_a_idx]}') = {reverse_count}")

In [ ]:
# Cell 2: function to show misclassified examples between two classes, then call it for the top confused pair
import matplotlib.pyplot as plt
import math
import random
import os

def show_misclassified_examples(paths, y_true, y_pred, class_names,
                                class_a_idx, class_b_idx,
                                num_examples=6, direction='both', seed=123):
    """
    Display misclassified images between class_a_idx and class_b_idx.

    - paths: array-like of file paths (same order as y_true and y_pred)
    - direction: 'both' (default) shows A->B and B->A; 'a_to_b' or 'b_to_a' restricts direction.
    """
    paths = np.array(paths)
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    if direction not in ('both', 'a_to_b', 'b_to_a'):
        raise ValueError("direction must be one of 'both', 'a_to_b', 'b_to_a'")

    if direction == 'both':
        mask = ((y_true == class_a_idx) & (y_pred == class_b_idx)) | ((y_true == class_b_idx) & (y_pred == class_a_idx))
    elif direction == 'a_to_b':
        mask = (y_true == class_a_idx) & (y_pred == class_b_idx)
    else:  # 'b_to_a'
        mask = (y_true == class_b_idx) & (y_pred == class_a_idx)

    indices = np.where(mask)[0]
    if len(indices) == 0:
        print("No misclassified examples found for this pair/direction.")
        return

    # sample indices
    random.seed(seed)
    if len(indices) > num_examples:
        indices = random.sample(list(indices), num_examples)
    else:
        indices = list(indices)

    n = len(indices)
    ncols = min(3, n)
    nrows = math.ceil(n / ncols)
    plt.figure(figsize=(4 * ncols, 3.5 * nrows))

    for i, idx in enumerate(indices):
        path = paths[idx]
        # Safe image read:
        try:
            img = plt.imread(path)
        except Exception as e:
            print(f"Could not read image at {path}: {e}")
            continue

        ax = plt.subplot(nrows, ncols, i + 1)
        # If image is float in [0,1] or uint8; imshow handles both.
        ax.imshow(img)
        ax.axis('off')
        true_name = class_names[int(y_true[idx])]
        pred_name = class_names[int(y_pred[idx])]
        # Show predicted probability if y_prob available
        prob_str = ""
        try:
            prob = y_prob[idx]
            pred_prob = prob[int(y_pred[idx])]
            prob_str = f"  ({pred_prob:.2f})"
        except Exception:
            pass
        ax.set_title(f"True: {true_name}\nPred: {pred_name}{prob_str}", fontsize=10)
    plt.tight_layout()
    plt.show()

# Call the function for the most confused pair found above
# Make sure test_paths exists and aligns with y_true/y_pred
try:
    test_paths  # noqa: F821
except NameError:
    raise NameError("test_paths not found in the environment. Ensure you have the test_paths array (file paths) available.")

show_misclassified_examples(
    paths=test_paths,
    y_true=y_true,
    y_pred=y_pred,
    class_names=class_names,
    class_a_idx=class_a_idx,
    class_b_idx=class_b_idx,
    num_examples=6,
    direction='both',
    seed=42
)

# Misclassification Visual Analysis

- **True class:** `<CLASS_A>`  
- **Predicted as:** `<CLASS_B>`  
- **Confusion count:** `<COUNT>`

## Observations (visual patterns)
- **Backgrounds:** many images share similar backgrounds (e.g., foliage, plain/white, indoor clutter).  
- **Color / tone:** dominant similar colors or low contrast across classes (e.g., brown/green, red/orange).  
- **Texture / pattern:** large uniform regions or repeating textures that hide fine details.  
- **Object scale / crop:** objects are often small, partially visible, or cropped at the image edges.  
- **Pose / viewpoint:** similar poses or orientations that remove distinguishing cues.  
- **Lighting / exposure:** underexposed or blown highlights reduce visible distinguishing features.  
- **Label ambiguity:** some images look ambiguous or contain multiple objects.

## Hypothesis
The model is confusing these classes because it relies on coarse color/texture and contextual background cues; when discriminative details are missing (small/cropped objects, poor lighting, or ambiguous labels) those low-level cues are shared across both classes causing misclassification.


- Inspect and clean ambiguous labels for these two classes.  
- Add targeted augmentations (color jitter, random crops, scaling) and more high-res examples.  
- Fine-tune higher backbone layers (low LR) so mid-level features can better separate the classes.

In [ ]:
# Save your trained model and class names from Colab
# Run this in the same Colab session where `model` and `class_names` exist.
import json
import os
from pathlib import Path

# Path to save model (you can also save to Google Drive: /content/drive/MyDrive/...)
SAVE_DIR = "/content/seg_model_saved"
os.makedirs(SAVE_DIR, exist_ok=True)

# Save the Keras model
# If your model variable is named `model` in Colab, this will save it.
print("Saving model to:", SAVE_DIR)
model.save(os.path.join(SAVE_DIR, "my_model.keras"), include_optimizer=False) # Added .keras extension
print("Model saved.")

# Save class names to JSON so the Streamlit app can load human-readable labels
class_names_path = Path(SAVE_DIR) / "class_names.json"
with open(class_names_path, "w") as f:
    json.dump(class_names, f)
print("Class names saved to:", class_names_path)

# Optional: copy to Google Drive (uncomment and adjust path if you want)
# drive_target = "/content/drive/MyDrive/seg_model_saved"
# !cp -r {SAVE_DIR} {drive_target}
# print("Copied saved model to Drive:", drive_target)

In [ ]:
import pickle
import os

# Define the path to save the pickle file
PICKLE_SAVE_PATH = "/content/seg_model_saved/my_model.pkl" # You can change the directory and filename

# Ensure the directory exists
os.makedirs(os.path.dirname(PICKLE_SAVE_PATH), exist_ok=True)

try:
    # Attempt to pickle the model
    with open(PICKLE_SAVE_PATH, 'wb') as f:
        pickle.dump(model, f)
    print(f"Model successfully pickled to: {PICKLE_SAVE_PATH}")
    print("Note: Pickling Keras models is not the recommended way to save them.")
    print("Consider using model.save() in the Keras native format (.keras) or TensorFlow SavedModel format instead.")

except Exception as e:
    print(f"An error occurred while pickling the model: {e}")
    print("Pickling Keras models can sometimes be problematic.")
    print("The recommended way to save your model is using model.save() as demonstrated in the cell above.")

# To load the model later:
# try:
#     with open(PICKLE_SAVE_PATH, 'rb') as f:
#         loaded_model = pickle.load(f)
#     print("Model successfully loaded from pickle.")
# except Exception as e:
#     print(f"An error occurred while loading the model from pickle: {e}")

In [ ]:
import tensorflow as tf

# Load the model from the correct path and filename
model = tf.keras.models.load_model("/content/seg_model_saved/my_model.keras")

# Save the model in the SavedModel format (optional, as it's already saved in .keras)
# model.save("seg_model_saved_tf")  # Saves as a SavedModel directory

print("Model loaded successfully.")